<p style="padding: 10px; border: 1px solid black;">
<img src="../images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>


# <a name="0">Building Applications with Bedrock Agents</a>
## <a name="0">Lab 1: Amazon Bedrock Agent Basics with AWS console</a>


This lab demonstrates how to build a foundation model (FM) powered customer service bot with Agents for Amazon Bedrock. Reference: [Build a foundation model (FM) powered customer service bot with agents for Amazon Bedrock](https://aws.amazon.com/blogs/machine-learning/build-a-foundation-model-fm-powered-customer-service-bot-with-agents-for-amazon-bedrock/).


It demonstrates how to:
1. Select the underlying foundation model (FM) for your agent 
2. Provide a clear and concise agent instruction 
3. Create an action group with an API Schema and a Lambda function 
4. Test and deploy your agent
5. Create a knowledge base using the AWS console

This folder contains the API schema, AWS Lambda function and SQLite database required for the use case.
You can find detailed instructions on how to create the Agent using the AWS console on the [Bedrock Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/a4bdb007-5600-4368-81c5-ff5b4154f518/en-US/90-agents)


We provide a step-by-step guide with building blocks to create a customer service bot. We use a text generation model ([Anthropic Claude V2](https://www.anthropic.com/index/claude-2)) and agents for Amazon Bedrock for this solution. Then we walk you through steps to create an agent for Amazon Bedrock.


<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This lab avoids auto-cleans up resources to allow you to create agent(s) with the provisioned resources. </h4>
    You can visit this section (<a href="#10"> Clean-up Resources</a>) to change the setting. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>

1. <a href="#1">How do agents work?</a>
2. <a href="#2">Lab Use Case: Retail agent solution overview</a>
3. <a href="#3">Script-based provisioning of agent support infrastructure like S3, Lambda, IAM resources</a>
4. <a href="#4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts</a>
5. <a href="#5"> Walkthrough of all infrastructure components created via script</a>
6. <a href="#6">[LAB TO-DO(s)] Setting up action groups</a>
7. <a href="#7">[LAB TO-DO(s)] Bedrock agent setup</a>
8. <a href="#8">[LAB TO-DO(s)] Testing agent</a>
9. <a href="#9">[LAB TO-DO(s)] Deploying agent</a>
10. <a href="#10">[LAB TO-DO(s)] Resource cleanup</a>
11. <a href="#11">Challenge exercise and lab quiz</a>
12. <a href="#12">[LAB EXTRA(S)]Create knowledge base via AWS console</a>
13. <a href="#12">Conclusion</a>
    

Please work on the steps from the top to the bottom of this notebook and don't skip sections as this could lead to error messages due to missing code/components.

---

You will be presented with two kinds of exercises throughout the notebook: Activities and challenges. <br/>

| <img style="float: center;" src="../images/activity.png" alt="Activity" width="125"/>| <img style="float: center;" src="../images/challenge.png" alt="Challenge" width="125"/>|
| --- | --- |
|<p style="text-align:center;">No coding is needed for an activity. You try to understand a concept, <br/>answer questions, or run a code cell.</p> |<p style="text-align:center;">Challenges are where you test your understanding by taking a short quiz.</p> |

---- 

In [1]:
%%capture
!pip3 install -r requirements.txt --quiet

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import boto3
from mlu_utils.agents_infra_utils_no_kb_setup_lab1 import *
from IPython.display import Markdown
from IPython.display import JSON

[2024-07-11 16:40:35,028] p17072 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [3]:
import ipywidgets as widgets
from IPython.display import JSON

infra_response_display = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
action_group_display = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
create_agent_display = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

### <a name="1">How do agents work?</a>
(<a href="#0">Go to top</a>)

#### ReAct Prompting

FMs determine how to solve user-requested tasks with a technique called [ReAct](https://react-lm.github.io/). It’s a general paradigm that combines reasoning and acting with FMs. ReAct prompts FMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while incorporating additional information into the reasoning. The structured prompts include a sequence of question-thought-action-observation examples.

* The question is the user-requested task or problem to solve. 
* The thought is a reasoning step that helps demonstrate to the FM how to tackle the problem and identify an action to take. 
* The action is an API that the model can invoke from an allowed set of APIs. 
* The observation is the result of carrying out the action.

#### Components in agents for Amazon Bedrock

Behind the scenes, agents for Amazon Bedrock automate the prompt engineering and orchestration of user-requested tasks. They can securely augment the prompts with company-specific information to provide responses back to the user in natural language. The agent breaks the user-requested task into multiple steps and orchestrates subtasks with the help of FMs. Action groups are tasks that the agent can perform autonomously. Action groups are mapped to an [AWS Lambda](https://aws.amazon.com/lambda/) function and related API schema to perform API calls. The following diagram depicts the agent structure.

<br/> <center><img src="./images/agents_components.png" alt="Agent Overview - what does agent need?: instructions, actions, knowledge bases and foundation model." width="500"/></center> <br/>

<br/> <center><img src="./images/agents-components1.png" alt="Agent components including action groups and Lambda functions. Action groups include API schemas and Lambda references. Lambda functions usually include different API methods." width="500" /></center> <br/>

---
### <a name="2">Lab Use Case: Retail agent solution overview</a>
(<a href="#0">Go to top</a>)

Agents can be used in a diverse set of use cases. From the simplest instruction only agent to complex assistants that combine action groups with knowledge bases, you can use the power of agents to quickly develop your Generative API application.

We use a shoe retailer use case to build the customer service bot. The bot helps customers purchase shoes by providing options in a humanlike conversation. Customers converse with the bot in natural language with multiple steps invoking external APIs to accomplish subtasks. The following diagram illustrates the sample process flow.



<br/> <center><img src="./images/sequence-flow-agents.png" alt="This figure is to show the sequence diagram to capture user and agent interaction for every conversation in the chat session to buy a pair of shoes. The bot gets customer information by asking the customer for their name, and then gets their details. Then the bot uses those details to find the right kinds of shoes for them, then checks its inventory database to be sure they're in stock, then creates a prompt that asks the FM to generate a response to the customer. Then the bot places an order on behalf of a customer." width="1000" height="1200" /></center> <br/>


The agent created can handle the following tasks:

* Get customer information based on the customer name
* Place an order on behalf of a customer
* Check inventory for available products


The following diagram depicts a high-level architecture of this solution.


<br/> <center><img src="./images/agents-arch-diagram.png" alt="This figure is to show the diagram that diagram depicts a high-level architecture of this solution.The user request is captured by agents to generate a plan and then it calls lambda to execute the API which can call any database, aws service like email or other applications." width="1000" height="1200" /></center> <br/>



1. You can create an agent with Bedrock-supported FMs such as Anthropic Claude V2.

2. Attach an API schema, residing in an [Amazon Simple Storage Service (Amazon S3)](https://aws.amazon.com/s3/) bucket, and a Lambda function containing the business logic to the agent. (Note: This is a one-time setup step.)

3. The agent uses customer requests to create a prompt using the ReAct framework. It, then, uses the API schema to invoke corresponding code in the Lambda function.

4. You can perform a variety of tasks including sending email notifications, writing to databases, triggering application APIs in the Lambda functions.

In this lab, we use the Lambda function to retrieve customer details, list shoes matching customer-preferred activity, and finally, place orders. Our code is backed by an in-memory SQLite database. You can use similar constructs to write to a persistent data store.

---
### <a name="3">Script-based provisioning of agent support infrastructure like S3, Lambda, IAM resources </a>
(<a href="#0">Go to top</a>)


Note you should use the following parameter values when creating and configuring your agent and linking it to Lambda, S3 and other resources. These have been created with the appropriate permissions via the `setup_agent_infrastructure` API inside `mlu_utils`. 

In [4]:
%%time
infra_response = setup_agent_infrastructure(schema_filename='customer_service_agent_openapi_schema.json', 
                                            kb_db_file_uri='retail_db', 
                                            lambda_code_uri='customer_service_agent_lambda_function.py')


[2024-07-11 16:40:35,736] p17072 {agents_infra_utils_no_kb_setup_lab1.py:57} INFO - random_uuid :: 281b904f-e1b8-40fb-b284-7fc43dded972 prefix_infra :: l2281b90 prefix_iam :: l2e1b8
[2024-07-11 16:40:36,000] p17072 {agents_infra_utils_no_kb_setup_lab1.py:124} INFO - kb_db_files_path :: retail_db kb_db_key :: kbdb_l2281b90
[2024-07-11 16:40:57,350] p17072 {agents_infra_utils_no_kb_setup_lab1.py:275} INFO - bucket_name :: l2281b90-agent-kb-339712993987 
 agent_name :: l2281b90-agent-kb 
 agent_alias_name :: l2281b90-workshop-alias 
 schema_key :: l2281b90-agent-kb-schema.json  


CPU times: user 72.7 ms, sys: 12.8 ms, total: 85.5 ms
Wall time: 21.6 s


In [5]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]



In [6]:
with infra_response_display:
    infra_response_display.clear_output()
    display(JSON(infra_response))


---
### <a name="4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts </a>
(<a href="#0">Go to top</a>)

In [7]:
%%time
## Display the infra_response values
display(infra_response_display)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

CPU times: user 1.77 ms, sys: 294 µs, total: 2.06 ms
Wall time: 1.66 ms


In [8]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '339712993987')

---
### <a name="5">Walkthrough of all infrastructure components created via script. This section is just for your understanding. </a>
(<a href="#0">Go to top</a>)

To implement the solution, you should have an AWS account and access to Amazon Bedrock with agents enabled. [Burner accounts](https://conduit.security.a2z.com/burner-accounts) can be used for this solution.


We will create two IAM roles. Update these roles to apply least-privilege permissions as discussed in [Security best practices](https://docs.aws.amazon.com/IAM/latest/UserGuide/best-practices.html#grant-least-privilege). Click [here](https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_service-with-iam-agent.html) to learn what IAM features are available to use with Agents for Amazon Bedrock.

1. `LambdaBasicExecutionRole` with S3 full access and CloudWatch access for logging.
2. `AmazonBedrockExecutionRoleForAgents` with S3 full access and Lambda full access. 

**Important:** Agents for Amazon Bedrock requires the role name to be prefixed by `AmazonBedrockExecutionRoleForAgents_*`

#### [Already Created via script] The IAM roles 
This step is optional. If you skip this step, you can create and use a new service role in the later step.
Or you can follow the step here and then later use an existing service role.

1. In your console, go to your [IAM Dashboard](https://us-east-1.console.aws.amazon.com/iam/).
2. Go to Policies in the right-hand side menu.
3. Create one policy named for example Bedrock-InvokeModel-Policy
   
   ```json
    
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicyProd",
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Resource": [
                    "arn:aws:bedrock:us-east-1::foundation-model/*"
                ]
            }
        ]
    }
   ```

4. Create one policy (name it something like Bedrock-S3-GetObject)
    ```json
    
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AmazonBedrockAgentS3PolicyProd",
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject"
                ],
                "Resource": [
                    "arn:aws:s3:::customer-support-bucket-agent-us-east-1-account/customer_service_agent_openapi_schema.json"
                ]
            }
        ]
    }
    ```


5. Create a role named AmazonBedrockExecutionRoleForAgents_<your alias> and attach the two policies we just created previously. 
    
    Click Add permissions, and then search for the policies ``AmazonBedrockAgentBedrockFoundationModelPolicyProd`` and ``AmazonBedrockAgentS3PolicyProd``. Then attach them to your role ``AmazonBedrockExecutionRoleForAgents_<your alias>``.

    


#### [Already Created via script] S3 Bucket and storing API schema and SQLite files on it 

Next, you need to create an S3 bucket to store the API Schema and SQLite files. To do so, first [navigate to `S3` on your AWS console](https://s3.console.aws.amazon.com/s3) .

Click on `Create bucket` and select a unique name for your bucket.

_Tip: Include your account number and region on the bucket name to ensure it is unique._

First select the AWS Region (us-east-1) and then add the bucket name. You can use the bucket name shown in the figure: ``customer-support-bucket-agent-<region>-<account>``. Please replace the region and account with your own account settings.


You will need to upload the files the **OpenAPI Schema** (customer_service_agent_openapi_schema.json) and **SQLite Database** (customer_service_agent_sqlite_db) to your S3 bucket. First click the S3 bucket that you just created and then click the upload button, select the files and then save them in the S3 bucket.




#### [Already Created via script] Creating Action Group Lambda Function


When creating an Action group, you need to define a Lambda function to program the business logic for your action group and to customize how you want the API response to be returned. You use the variables from the input event to define your functions and return a response to the agent. To write your function, you will need to understand the format of the input event and expected response. You must attach a resource-based policy to your Lambda function. For more information about resource-based policies in Lambda, see [Using resource-based policies for Lambda](https://docs.aws.amazon.com/lambda/latest/dg/access-control-resource-based.html) .


This function uses a SQLite database that will be stored in your S3 bucket. We will copy the files to a newly created bucket in the next step.

To create the Lambda function via AWS console, open your console and navigate to AWS Lambda 

Create a new Lambda function named ``customer_support_bot_agent``, with `Python 3.12` as runtime language and `x86_64` as the architecture. Other settings we can keep as the default values.


Now, you should get a message that says "Successfully created the function customer_support_bot_agent."

Replace the function code for the Lambda function (lambda_function.py) with the below code:

<!-- <br/> <center><img src="./images/lambda-deploy.png" alt="This figure is to show that where you can replace the lambda function code for your agent." width="800" /></center> <br/> -->


For the use case, let's use the following code to replace the function code for the Lambda function (lambda_function.py ):

```python

import json
import boto3
import sqlite3
from datetime import datetime
import logging
import os

logger = logging.getLogger()
logger.setLevel(logging.INFO)

s3 = boto3.client('s3')
bucket = os.environ['S3_BUCKET']  # Name of bucket with data file and OpenAPI file
db_name = 'customer_service_agent_sqlite_db'  # Location of data file in S3
local_db = '/tmp/csbot.db'  # Location in Lambda /tmp folder where data file will be copied

# Download data file from S3 to your local lambda storage
print(bucket, db_name, local_db)
s3.download_file(bucket, db_name, local_db)

cursor = None
conn = None


def get_named_parameter(event, name):
    """
    Function that gets the parameter 'name' from the lambda event object
    Args:
        event: lambda event
        name: name of the parameter to return
    Returns:
        parameter value
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_named_property(event, name):
    """
    get the named property 'name' from the lambda event object
    Args:        
        event: lambda event
        name: name of the named property to return
    Returns:
        named property value
    """
    return next(item for item in event['requestBody']['content']['application/json']['properties'] if item['name'] == name)['value']


def load_data():
    """
    Initial data load and SQLite3 cursor creation
    Returns:
        a cursor for the connection
    """
    # load SQL Lite database from S3
    # create the db
    global conn
    conn = sqlite3.connect(local_db)
    c = conn.cursor()
    logger.info('Completed initial data load ')
    return c


def return_customer_info(cust_name, c):
    """
    Function returns all customer info for a particular cust_name
    Args:
        cust_name (str): customer name to query database
        c: a cursor for the connection
    Returns:
        customer information
    """
    query = 'SELECT customerId, customerName, Addr1, Addr2, City, State, Zipcode, PreferredActivity, ShoeSize, OtherInfo FROM CustomerInfo WHERE customerName LIKE "%' + cust_name + '%"'
    c.execute(query)
    resp = c.fetchall()
    # adding column names to response values
    names = [description[0] for description in c.description]
    val_dict = {}
    index = 0
    for name in names:
        val_dict[name] = resp[0][index]
        index = index + 1
    logger.info('Customer Info retrieved')
    return val_dict


#
def return_shoe_inventory(c):
    """
    Function returns shoes inventory
    Args:
        c: a cursor for the connection
    Returns:
        Shoes inventory
    """
    query = 'SELECT ShoeID, BestFitActivity, StyleDesc, ShoeColors, Price, InvCount FROM ShoeInventory'
    c.execute(query)
    resp = c.fetchall()

    # adding column names to response values
    names = [description[0] for description in c.description]
    val_dict = []
    index = 0
    for item in resp:
        interim_dict = {}
        for name in names:
            interim_dict[name] = item[index]
            index = index + 1
        index = 0
        val_dict.append(interim_dict)
    logger.info('Shoe info retrieved')
    return val_dict


def place_shoe_order(shoe_id, cust_id, c, db_conn):
    """
    function places order -- reduces shoe inventory, updates order_details table --> all actions resulting from a shoe purchase
    Args:
        shoe_id (int): shoe id
        cust_id (int): customer id
        c: a cursor for the connection
        db_conn: database connector
    Returns:
        c: a cursor for the connection
    """
    query = 'Update ShoeInventory set InvCount = InvCount - 1 where ShoeID = ' + str(shoe_id)
    c.execute(query)

    today = datetime.today().strftime('%Y-%m-%d')
    query = 'INSERT INTO OrderDetails (orderdate, shoeId, CustomerId) VALUES ("' + today + '",' + str(shoe_id) + ',' + str(cust_id) + ')'
    c.execute(query)
    db_conn.commit()

    # Writing updated db file to S3 and setting cursor to None to force reload of data
    s3.upload_file(local_db, bucket, db_name)
    c = None
    logger.info('Shoe order placed')
    return c


def lambda_handler(event, context):
    responses = []
    global cursor
    if cursor is None:
        cursor = load_data()
    shoe_id = ''
    api_path = event['apiPath']
    logger.info('API Path')
    logger.info(api_path)
    body = ""
    if api_path == '/attributes/{CustomerName}':
        c_name = get_named_parameter(event, "CustomerName")
        body = return_customer_info(c_name, cursor)
    elif api_path == '/transaction':
        shoe_id = get_named_parameter(event, "ShoeID")
        cid = get_named_parameter(event, "CustomerID")
        parameters = event['parameters']
        body = place_shoe_order(shoe_id, cid, cursor, conn)
    elif api_path == '/assets':
        body = return_shoe_inventory(cursor)
    else:
        body = {"{} is not a valid api, try another one.".format(api_path)}

    response_body = {
        'application/json': {
            'body': json.dumps(body)
        }
    }

    action_response = {
        'actionGroup': event['actionGroup'],
        'apiPath': event['apiPath'],
        'httpMethod': event['httpMethod'],
        'httpStatusCode': 200,
        'responseBody': response_body
    }

    responses.append(action_response)

    api_response = {
        'messageVersion': '1.0',
        'response': action_response}

    return api_response
```

Next you need to create an environmental variable that has the name of your recently created bucket. To do so, return to your Lambda function and navigate to `Configuration > Environment variables`. 

In the Environment variables menu, click the Edit button.


Then create a new environmental variable called `S3_BUCKET` and having as value the name of the bucket you just created. Clicking Save when done.

If you've successfully created this environment variable, you should see one named S3_BUCKET with your bucket's name as its value here.

Now you need to allow your Lambda function to access S3 location containing the database file. To do so, navigate to the Lambda IAM role and add attach the `AmazonS3FullAccess` policy 

* Click on the IAM role `customer_support_bot_agent-role-****` 
* Click `Add permissions` button
* Select `Attach policies`
* Search for the policy `AmazonS3FullAccess`
* Click `Add permissions` button


<!-- <br/> <center><img src="./images/lambda-role-attach-policy1.png" alt="This figure is to show when the policy was successfully attached to the role." width="800" /></center> <br/> -->
You should now see a message that says, "Policy was successfully attached to role."




#### Creating OpenAPI Schema file
When you create an action group, you must define the APIs that the agent can invoke with an OpenAPI schema in JSON or YAML format. You can create OpenAPI schema files yourself and upload them to Amazon S3 buckets, or you can use the OpenAPI text editor in order to easily validate your schema. The text editor is available after you create the agent. You can use the text editor when you add an action group to an agent or edit an existing one (for more information, see [Manage your agent](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-edit.html) ).
* **Pro-tip**: Avoid using path names and operation IDs that may be exact matches to words that would be used in the user request or agent response to avoid REDACTIONS on your agent’s answers.
* You can also create your OpenAPI schema file directly on Bedrock by using the console editor - To do so, create your Agent without an action group and once the Agent is available, edit it to include an action group with an OpenAPI schema

The following is the general format of an OpenAPI schema for an action group.

```json
{
    "openapi": "3.0.0",
    "paths": {
        "/path": {
            "method": {
                "description": "string",
                "operationId": "string",
                "parameters": [ ... ],
                "requestBody": { ... },
                "responses": { ... }
           }
       }
    }
}
```
Minimally, each method requires:

- description – A description of the API operation. Use this field to inform the agent when this API should be called and what it does.
- responses – The properties defined for the operation response are not just used for constructing prompts. They are used for accurately processing the results of an API call, and for determining a proper set of steps for carrying out a task. By knowing the response coming back from one operation, the agent can know that those values can be used as inputs for subsequent steps in the process.

For this use case, you will use the API Schema available in the lab1 folder ``customer_service_agent_openapi_schema.json``. It consists of 3 paths:

- `/attributes/{CustomerName}`: Based on provided customer name, return customer information like customer ID, preferred activity and others
- `/transaction`: Place an order for a shoe by creating an Order record and updating inventory in the database
- `/assets`: Checks inventory for shoes and returns all available information about available shoes, including shoe ID, shoe colors, inventory, best fit activity, style description and price


<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>[LAB TO-DOs START] This NEXT section is what you will be creating using the AWS console in this Lab. </h4>
    You will need to use the previously created infrastructure components like S3, Lambda, IAM roles setup in the cells above to create agents and associate them with the respective components. <br/>
</div>
<br/>

---
### <a name="6">[LAB TO-DO(s)] Setting up Action Groups</a>
(<a href="#0">Go to top</a>)

<a href="#4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts</a>

Action Group Description: An overview of the actions provided, helping the agent know when this action group is relevant.


Before you create your agent, you should set up action groups and knowledge bases that you want to add to your agent.

* **Action groups** define the tasks that you want your agent to help customers carry out.

* **Knowledge bases** provide a repository of information that the agent can query to answer customer queries and improve its generated responses. For more information, see knowledge base for Amazon Bedrock.

An action group consists of the following components, which you will set up: 

* An **OpenAPI schema** that define the APIs that your action group should call. Your agent uses the API schema to determine the fields it needs to elicit from the customer to populate for the API request.
    * Rich description of each action, so agents know when to use it, and exactly how to call it and use results
    * Operation name, input parameters, data types, response details
    * Language agnostic
    * Uses industry-standard schema
    * OpenAPI schema file in JSON or YAML format
    * Requires OpenAPI version “3.0.0” or higher
 <p>    
    
* A **Lambda function** that defines the business logic for the action that your agent will carry out. You need to create a Lambda function to process your API paths. Create Lambda function and add Lambda invocation permissions to your agent by attaching a resource-based policy to your Lambda.
    * Process input event from Amazon Bedrock to identify requested action
    * Perform Agent action
    * Return Lambda response with output from the action
    * Contains business logic, or wraps microservices, databases, or tools
    * Serverless, scalable, secure
    * Flexible programming language choice (Python, C#, JavaScript, Java, ...)

You will need to create a new action group on your Agent and link it with your API schema and your Lambda function.

In [9]:
action_group_setup_params = {"agent_name": agent_name, "lambda_name": lambda_name, "s3_bucket_name": bucket_name, "region": region, "account_id": account_id}

with action_group_display:
    action_group_display.clear_output()
    display(JSON(action_group_setup_params))

----
#### 1. [LAB TO-DO] Permissions to create and manage an agent inside the `Lambda configuration`
Add permissions to create and invoke Lambda functions by attaching a resource-based policy to your Lambda.


#### TODO: 
Add resource-based policy for Bedrock to invoke the model. In the `Lambda function`, first go to the`Configuration/Permissions` panel and then scroll down to `Resource-based policy statements` and click on `Add permissions`, choose `AWS Service` > `other` from that dropdown of listed AWS services.

 1. Set statement-id can be anything meaningful like `my-custom-id-0001`
 2. Set principal to `bedrock.amazonaws.com` 
 3. Set source ARN to `arn:aws:bedrock:<REGION>:<ACCOUNT_NUMBER>:agent/*` 
 4. Set Action to `lambda:InvokeFunction`
and click on `Save` to continue

Example:


Template:
```json
{
  "Version": "2012-10-17",
  "Id": "default",
  "Statement": [
    {
      "Sid": "<statement-name>",
      "Effect": "Allow",
      "Principal": {
        "Service": "bedrock.amazonaws.com"
      },
      "Action": "lambda:InvokeFunction",
      "Resource": "arn:aws:lambda:<region>:<account-id>:function:<function-name>",
      "Condition": {
        "ArnLike": {
          "AWS:SourceArn": "arn:aws:bedrock:<region>:<account-id>:agent/*"
        }
      }
    }
  ]
}
```


#### After adding and saving the resource based policy, it will reflect in the lambda function settings 


----
### [Reading] Link between the action group(s) and Lambda(s)

----
#### Creating an Action Group - Input Event
Process the input event from Amazon Bedrock to identify the requested action

    "messageVersion": "1.0", 
    "agent": { … },  ->   Agent name, id, alias and version
    "inputText": "string",  ->   User input for the conversation
    "sessionId": "string",  ->   Unique identifier of agent session 
    "actionGroup": "string",  ->   Action Group name
    "apiPath": "string",  ->   The path to the API operation
    "httpMethod": "string",  ->   HTTP method of the API operation (“GET”, “POST”, …)
    "parameters": [ … ],  ->   List of parameter objects. Each object contains the name, type, and value of a parameter
    "requestBody": { … },  -> Request body and its properties
    "sessionAttributes": { … },
    "promptSessionAttributes": { … }


<!-- <br/> <center><img src="./images/input-event.png" alt="This figure is show the description of each parameter in an input event." width="800" /></center> <br/> -->


----
#### Creating an Action Group – Processing Inputs
Process input event from Amazon Bedrock to identify requested action

Examples of event processing:

* Getting event API path: 
```python
def lambda_handler(event, context):
    api_path = event['apiPath']
```
* Process data shared via GET HTTP method: 
```python
def get_named_parameter(event, name):
    return next(i for i in event['parameters'] if i['name'] == name)['value']
```
* Process data shared via POST HTTP method: 
```python
def get_named_property(event, name):
    prop = event['requestBody']['content']['application/json']['properties']
    return next(i for i in prop if i['name'] == name)['value']
```


----
#### Creating an Action Group – Creating Lambda Function

Creating a Lambda function to perform agent actions -
The action performed by the Lambda function is flexible. You can invoke other APIs, another agent or any AWS service as long as your Lambda function has the correct permissions for it.

```python
    api_path = event['apiPath']
    body = ""
    if api_path == '/attributes/{CustomerName}':
        c_name = get_named_parameter(event, "CustomerName")
        body = return_customer_info(c_name, cursor)
    elif api_path == '/transaction':
        shoe_id = get_named_parameter(event, "ShoeID")
        cid = get_named_parameter(event, "CustomerID")
        parameters = event['parameters']
        body = place_shoe_order(shoe_id, cid, cursor, conn)
    elif api_path == '/assets':
        body = return_shoe_inventory(cursor)
    else:
        body = {
            "{}::{} is not a valid api, try another one.".format(
                action_group,
                api_path
            )
        }
```
---
**Notes:** 

The Lambda functions defined for action groups can have different implementations based on the specific requirements and use cases. These are some potential variances:

- Language: Lambda functions can be written in various languages supported by AWS Lambda, such as Python, Node.js, Java, C#, Go, and more. The choice of language may depend on the development team's expertise or the specific requirements of the action being performed.

- Synchronous vs. Asynchronous: The Lambda function can be implemented to run synchronously, where it waits for the API invocation to complete and returns the result. Alternatively, it can be implemented asynchronously, where it initiates the API call and returns immediately, allowing the agent to continue its orchestration while the API call is processed in the background.

- Error Handling: Different strategies can be employed for handling errors and exceptions that may occur during the API invocation. The Lambda function can implement retries, fallback mechanisms, or logging and monitoring capabilities.

- Data Transformation: Depending on the API being invoked and the data formats involved, the Lambda function may need to perform data transformations, such as converting between JSON, XML, or other formats, or mapping data structures to match the expected input/output formats.

- Authentication and Authorization: If the API invocation requires authentication or authorization, the Lambda function may need to handle token generation, credential management, or other security-related tasks.

- External Dependencies: The Lambda function may need to interact with external services, databases, or other resources beyond the API invocation itself. This could involve making additional calls, querying data stores, or integrating with third-party services.

- Parallelization and Batching: For scenarios involving multiple API invocations or large data sets, the Lambda function could implement parallelization or batching strategies to improve performance and efficiency.

- Logging and Monitoring: Different logging and monitoring strategies can be implemented within the Lambda function to track execution, capture metrics, or integrate with observability tools for debugging and troubleshooting purposes.

- These are just a few examples of potential variances in Lambda implementations for action groups. The specific implementation details will depend on the requirements of the actions being performed, the APIs being invoked, and the overall architecture and design decisions made by the development team.


---
### <a name="7">[LAB TO-DO(s)] Bedrock agent setup</a>
(<a href="#0">Go to top</a>)

<a href="#4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts</a>

Now that you have all the pre-requisites in place, let's create the Bedrock agent. To do so, first navigate to Bedrock on your AWS console.

#### Create an Agent for Amazon Bedrock

To create an Agent, open the [Amazon Bedrock console](https://console.aws.amazon.com/bedrock/home) and choose **Agents** in the left navigation pane from the Orchestration drop-down menu. 


Then select **Create Agent**.


In [10]:
print(f"USE agent_name ::: {agent_name}")

USE agent_name ::: l2281b90-agent-kb


Use the Agent name provided and set a description to your Agent. Use the `Create and use a new service role` option for your IAM role.


#### Setting Agent's instructions

We now need to set clear and specific instructions for the task the Agent will perform. Provide clear and precise instructions to the Agent about what tasks to perform and how to interact with the users. Copy the following text on your instructions field. 

**Select a foundation model:** In the **Select model** screen, select `Claude V3 Haiku` as your foundation model

In [11]:
print(f"Use lambda_name :: {lambda_name} Use S3 bucket_name :: {bucket_name}")

Use lambda_name :: l2281b90-agent-kb-339712993987 Use S3 bucket_name :: l2281b90-agent-kb-339712993987


```text
You are an agent that helps customers purchase shoes. You can 
1/ Retrieve customer details like customer ID and preferred activities based on the customer name. 
2/ Check shoe inventory and find the best fit product to match customer's preferred activities. 
3/ Generate a response with shoe’s ID, style, description and colors based on shoe’s inventory details. 
4/ If multiple matches exist, replay all of them to the customer. 
5/ After the customer indicates that they would like to order the shoe, you can use the shoe’s ID corresponding to the customer choice and the customer ID from the customer details retrieved to place an order for the requested shoe.
If no clear instructions are provided, ask for the customer's name in order to retrieve their details
```

##### Note: click `Save` button first before moving to next section. 
Otherwise you might get the following `Error message You must save your agent with an Agent Resource Role defined before adding an action group that uses an API schema from S3.` when trying to create action groups.

#### Creating Agent's action group
<!-- 
An action is a task the agent can perform by making API calls. A set of actions comprise an action group. You provide an API schema that defines all the APIs in the action group. You must provide an API schema in the [OpenAPI schema](https://swagger.io/specification/) JSON format. The Lambda function contains the business logic needed to perform API calls. You must associate a Lambda function to each action group. -->


To create the action group that connects the Lambda function and OpenAPI schema created before. You should also set a description for your action group that informs the Agent about the functionalities available. Give the action group a name and a description for the action. Copy the following text on the description field:

```text
Action group that provides agent functionality for 1/ get customer information 2/ place shoe order 3/ check shoe inventory
```

Choose `Define with API schemas` option in `Action Group Type` and then :

1. From `Action group invocation`, Select the option `existing Lambda function` and choose the lambda name given above.
2. From `Action group schema`, choose `select an existing API schema` and browse the file from s3 uri `s3://{bucket_name}/*-agent-kb-schema.json`(bucket name above), which is the `API schema` file we automatically uploaded to the newly created S3 bucket, and select **Next**.


As you are not adding a knowledge base at the moment, you can skip this step and save your Agent.

In the final step, review the Agent configuration and use **`Save` or `Save and exit` button to save your new Agent**.



---
### <a name="8">[LAB TO-DO(s)] Testing agent</a>
(<a href="#0">Go to top</a>)

<a href="#4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts</a>

**Test the Agent:** 

Now that you've created your first Agent, it is time to `prepare` the Agent for testing.

After the Agent is prepared, a dialog box shows the Agent overview along with a working draft. The Amazon Bedrock console provides a UI to test your Agent.

When you first create an Agent, you will have a working draft version and a `TestAlias` alias that points to it. The working draft is a version of the Agent that you can use to iteratively build the Agent. By default, you can interact with the working draft with the `TestAlias`, You can also select a different alias to test. In the test window, you can choose to show the trace for each response. The trace shows the agent's reasoning process, step-by-step, and is a useful tool for debugging your agent. To learn more about the trace, see [Trace events](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) .

Let's ask some questions to the agent. For example, we can ask `hello, I am John Doe, I want to buy running shoes` first. Then the agent will reply: `Here are the running shoe options that would be a good fit for you, John Doe: 1. ShoeID 1: Lightweight mesh runners in Red and Blue, $129.98 2. ShoeID 4: Breathable mesh runners in Pink and Purple, $67.35 3. ShoeID 7: Cushioned running shoes in Black and Lime, $75.99 4. ShoeID 10: Lightweight racing flats in Yellow and Orange, $129.99 Please let me know which of these shoes you would like to order and I'll place the order for you.`. Then we can ask for more details for any particular ShoeID like `Give me more details on Shoe ID 7`. If happy with the details for the Shoe, we can order the shoe like this `Place the order for Shoe ID 7`.


To understand the Agent's reasoning you can use the `Show trace >` link



You can now update your Agent instruction, OpenAPI Schema and Lambda functions and use the Amazon Bedrock console testing functionalities to test the reactions of your Agent.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="../images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Try different prompts and observe the responses generated by the model. You can now update your agent instruction, OpenAPI Schema and Lambda functions and use the Amazon Bedrock console testing functionalities to test the reactions of your Agent.</p>
    <br>
</div>


---
### <a name="9">[LAB TO-DO(s)] Deploying Agent</a>
(<a href="#0">Go to top</a>)

<a href="#4">[USE THESE VALUES FOR All LAB TO-DO(s) SECTION] Parameter values for infrastructure components like S3, Lambda, IAM, Agent created via scripts</a>

**Deploy:** After successful testing, you can deploy your Agent.

After you have sufficiently iterated on your working draft and are satisfied with the behavior of your Agent, you can set it up for deployment and integration into your application by creating aliases of your Agent. To deploy an Agent in your application, you must create an alias. Amazon Bedrock then automatically creates a version for that alias.

To deploy your Agent, you need to create an alias. During alias creation, Amazon Bedrock automatically creates a version of your Agent. The alias points to this newly created version. You can point the alias to a previously created version if necessary. You then configure your application to make API calls to that alias.

The version is a snapshot that preserves the resource as it exists at the time it was created. We can keep modifying the working draft and create new aliases (and consequently, versions) of your agent as necessary. In Amazon Bedrock, you create a new version of your Agent by creating an alias that points to the new version by default. Amazon Bedrock creates versions in numerical order, starting from 1. Because a version acts as a snapshot of your Agent at the time you created it, it is immutable.

Aliases let you efficiently switch between different versions of your Agent without requiring the application to keep track of the version. For example, you can change an alias to point to a previous version of your Agent if there are changes that you need to quickly revert.

The working draft version is `DRAFT` and the alias that points to it is the `TestAlias`.

To create a new Alias, you need to navigate to the agent page and click on the `Create Alias` button.


Provide a new Alias and click on `Create Alias`. We can set the alias name as `customer-support-bot-v1`. The description part is optional. Or we can write `version 1.0 of customer support bot` to describe the alias.



---
### <a name="10">[LAB TO-DO(s)] Resource Cleanup</a>
(<a href="#0">Go to top</a>)

Auto-clean up resources

In [12]:
# This avoids auto-cleanup so that Agents can be setup with the resources created
raise Exception('Avoiding Auto-Cleanup of Bedrock Agent Resources')


Exception: Avoiding Auto-Cleanup of Bedrock Agent Resources

In [ ]:
%load_ext autoreload
%autoreload 2
from mlu_utils.agents_infra_utils_no_kb_setup_lab1 import *
cleanup_infrastructure_lab1(lambda_name, lambda_role_name, bucket_name, schema_key, agent_bedrock_policy, agent_s3_schema_policy)

---
### <a name="11">Challenge Exercise and Lab Quiz</a>
(<a href="#0">Go to top</a>)

### Challenge Exercise 

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="../images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Now, try to ask more questions to your built Bedrock Agent and see what response you will get. Observe what works and what does not.</p>
    <br>
</div>



### Lab Quiz

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="../images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Answer the following questions to test your understanding of using Agents for Amazon bedrock.</p>
    <br>
</div>

In [ ]:
from mlu_utils.agents_quiz import *

lab1_question1

In [ ]:
lab1_question2

---
### <a name="12">[LAB EXTRA(S)]Create knowledge base via AWS console</a>
(<a href="#0">Go to top</a>)

In your AWS console under Amazon Bedrock, in the left hand menu bar, under the `Builder Tools` sub-heading you will find the `knowledge base` menu.

Step 1: You can click on the `Knowledge Bases` link to refresh the page and then proceed to `Create Knowledge Base` button.


Step 2: You will need to provide knowledge base details, including name, description and IAM service role


Step 3: You can either create a new IAM service role or use an existing service role


Step 4: Next, set up a data source



Step 5: Next, configure vector store using an embedding model and create a new vector store by default


Step 6: More optional settings are available under `configure vector store`


Step 7: Review and create the knowledge base


Step 8: After knowledge base has been created, you can click sync button to create embeddings from the documents at the S3 bucket


Step 9: You will see the status changes to ready from syncing after the syncing is finished


---
### <a name="13">Conclusion</a>
(<a href="#0">Go to top</a>)

Agents for Amazon Bedrock can help you increase productivity, improve your customer service experience, or automate DevOps tasks. In this lab, we showed you how to set up agents for Amazon Bedrock to create a customer service bot.

We encourage you to learn more by reviewing the [User Guide ](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html) and [additional features](https://aws.amazon.com/bedrock/knowledge-bases/) of agents for Amazon Bedrock. In the next lab 2a, we show you the boto3 API version of the lab.


<p style="padding: 10px; border: 1px solid black;">
<img src="../images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>

# Thank you!